# Modern AI Landscape - Practical Examples

This notebook demonstrates the key concepts from Session 2 through **hands-on examples**:

1. **Classical vs. Modern AI**: See the difference between predictive models and generative AI
2. **Prompt Engineering**: Learn how to effectively interact with LLMs
3. **RAG (Retrieval-Augmented Generation)**: Build a simple document Q&A system

By the end, you'll understand how modern AI works in practice and how it differs from traditional machine learning.

---
## Example 1: Classical AI vs. Modern AI

Let's compare how classical AI and modern AI solve different types of problems using the **same dataset** - customer reviews.

In [ ]:
# Setup: Install required libraries
!pip install -q scikit-learn openai pandas

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import openai
import os

# Sample customer reviews dataset
reviews = [
    "This product is amazing! Best purchase ever.",
    "Terrible quality. Broke after one week.",
    "Good value for money, works as expected.",
    "Waste of money. Very disappointed.",
    "Excellent customer service and fast shipping!",
    "Poor design, hard to use.",
    "Love it! Highly recommend to everyone.",
    "Not worth the price. Low quality materials."
]

# Labels: 1 = Positive, 0 = Negative
labels = [1, 0, 1, 0, 1, 0, 1, 0]

df = pd.DataFrame({'review': reviews, 'sentiment': labels})
print("Dataset:")
print(df)

### Classical AI Approach: Sentiment Classification

Traditional ML uses **structured features** (word frequencies) to classify text into predefined categories.

In [ ]:
# CLASSICAL AI: Train a sentiment classifier

# Step 1: Convert text to numerical features (TF-IDF)
vectorizer = TfidfVectorizer(max_features=20)
X = vectorizer.fit_transform(reviews)

# Step 2: Split data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.25, random_state=42)

# Step 3: Train logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Step 4: Predict on new review
new_review = ["Fantastic product, exceeded expectations!"]
new_review_vec = vectorizer.transform(new_review)
prediction = classifier.predict(new_review_vec)[0]

print(f"\n--- CLASSICAL AI RESULT ---")
print(f"Review: {new_review[0]}")
print(f"Prediction: {'Positive' if prediction == 1 else 'Negative'}")
print(f"\nKey insight: Classical AI assigns to predefined categories (Positive/Negative).")
print(f"It cannot explain WHY or generate new content.")

### Modern AI Approach: Generative Understanding

Modern AI (LLMs) can not only classify but also **explain reasoning** and **generate responses**.

**Note**: You'll need an OpenAI API key. For this demo, we'll show the prompt structure. In practice, you'd call the API.

In [ ]:
# MODERN AI: Use LLM for sentiment analysis + explanation

# Construct prompt for LLM
new_review = "Fantastic product, exceeded expectations!"

prompt = f"""Analyze the sentiment of this customer review and explain your reasoning.

Review: "{new_review}"

Provide:
1. Sentiment (Positive/Negative/Neutral)
2. Confidence score (0-100%)
3. Key phrases that influenced your decision
4. Suggested response to the customer
"""

print("--- MODERN AI PROMPT ---")
print(prompt)
print("\n--- SIMULATED LLM RESPONSE ---")
print("""
1. Sentiment: Positive
2. Confidence: 95%
3. Key phrases:
   - "Fantastic" - strong positive adjective
   - "exceeded expectations" - indicates superior performance
4. Suggested response:
   "Thank you for your wonderful feedback! We're thrilled the product 
   exceeded your expectations. We'd love to hear more about your experience!"
""")

print("\n🔑 KEY DIFFERENCE:")
print("Classical AI: Predicts category (Positive)")
print("Modern AI: Understands context, explains reasoning, generates personalized responses")

---
## Example 2: Prompt Engineering Fundamentals

The quality of LLM outputs depends heavily on **how you prompt them**. Let's see the difference between basic and advanced prompting.

In [ ]:
# Scenario: Generate a product description for an e-commerce site

product_features = {
    'name': 'UltraLight Camping Tent',
    'features': ['2-person capacity', 'Weighs 3 lbs', 'Waterproof', 'Easy setup'],
    'price': '$179.99'
}

# ❌ POOR PROMPT: Vague, no context
poor_prompt = f"Write about {product_features['name']}"

print("--- POOR PROMPT ---")
print(poor_prompt)
print("\nSimulated Output:")
print("The UltraLight Camping Tent is a tent. It's good for camping.")
print("\n⚠️ Problem: Too generic, not persuasive, no structure.\n")

print("="*60 + "\n")

# ✅ GOOD PROMPT: Specific role, format, constraints, examples
good_prompt = f"""You are a professional e-commerce copywriter specializing in outdoor gear.

Task: Write a compelling product description.

Product: {product_features['name']}
Features: {', '.join(product_features['features'])}
Price: {product_features['price']}

Requirements:
- Length: 100-150 words
- Tone: Adventurous yet professional
- Include benefits (not just features)
- Use persuasive language
- End with a call-to-action

Format:
[Engaging headline]
[Body paragraph 1-2]
[Call to action]
"""

print("--- GOOD PROMPT ---")
print(good_prompt)
print("\nSimulated Output:")
print("""
**Adventure Awaits Without the Weight**

Escape to the wilderness with the UltraLight Camping Tent, engineered for 
adventurers who refuse to compromise. At just 3 pounds, this 2-person tent 
disappears in your pack, letting you trek farther and explore deeper. 
Waterproof construction keeps you dry through mountain storms, while the 
intuitive setup gets you sheltered in minutes—more time enjoying the sunset, 
less time wrestling with poles. Whether you're conquering a multi-day trail 
or weekend camping with friends, this tent delivers comfort without bulk.

At $179.99, invest in gear that matches your ambition. Order now and get 
free shipping on your next adventure.
""")

print("\n✅ Result: Specific, persuasive, follows structure, includes benefits!")

### Prompt Engineering Best Practices

**Key elements of effective prompts:**
1. **Role**: Define who the AI should be ("You are a...")
2. **Context**: Provide necessary background information
3. **Task**: Be specific about what you want
4. **Constraints**: Set length, tone, format requirements
5. **Examples**: Show the AI what good output looks like (few-shot learning)
6. **Output Format**: Specify structure (bullet points, JSON, etc.)

In [ ]:
# Few-Shot Learning Example
# Teaching the LLM by example rather than explicit rules

few_shot_prompt = """
Extract key information from product reviews in JSON format.

Example 1:
Review: "Battery life is amazing, lasts 3 days! But screen is too small."
Output: {"pros": ["Long battery life (3 days)"], "cons": ["Small screen"], "rating_estimate": 4}

Example 2:
Review: "Terrible quality. Broke after 1 week. Customer service was rude."
Output: {"pros": [], "cons": ["Poor quality", "Broke quickly", "Bad customer service"], "rating_estimate": 1}

Now extract from this review:
Review: "Great camera quality and fast charging. Wish it had more storage."
Output:
"""

print("--- FEW-SHOT LEARNING PROMPT ---")
print(few_shot_prompt)
print("\nExpected Output:")
print('{"pros": ["Great camera quality", "Fast charging"], "cons": ["Limited storage"], "rating_estimate": 4}')
print("\n💡 The AI learns the pattern from examples without explicit programming!")

---
## Example 3: RAG (Retrieval-Augmented Generation) Simplified

RAG combines **information retrieval** with **LLM generation** to answer questions about specific documents. This is how ChatGPT can answer questions about *your* company docs.

In [ ]:
# Simulate a simple RAG system for company policy documents
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Company knowledge base (simplified documents)
documents = [
    "Our return policy allows customers to return items within 30 days of purchase for a full refund. Items must be in original packaging and unused.",
    "Shipping is free for orders over $50. Standard shipping takes 3-5 business days. Express shipping is available for $15 and takes 1-2 days.",
    "Customer support is available Monday-Friday 9am-6pm EST. You can reach us by email at support@company.com or call 1-800-SUPPORT.",
    "We offer a 1-year warranty on all products. The warranty covers manufacturing defects but does not cover damage from misuse or accidents.",
    "Account passwords must be reset every 90 days for security. Passwords must be at least 12 characters and include numbers and symbols."
]

print("📚 KNOWLEDGE BASE (5 documents)")
for i, doc in enumerate(documents, 1):
    print(f"{i}. {doc[:60]}...")
print()

In [ ]:
# STEP 1: Vectorize documents (create embeddings)
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(documents)

print("STEP 1: Documents converted to vectors ✓")
print(f"Vector dimensions: {doc_vectors.shape[1]}\n")

# STEP 2: User asks a question
user_question = "How long do I have to return a product?"
print(f"USER QUESTION: '{user_question}'\n")

# STEP 3: Convert question to vector
question_vector = vectorizer.transform([user_question])

# STEP 4: Find most similar documents (retrieval)
similarities = cosine_similarity(question_vector, doc_vectors)[0]
most_relevant_idx = np.argmax(similarities)
most_relevant_doc = documents[most_relevant_idx]

print("STEP 2-4: Retrieved most relevant document:")
print(f"Document #{most_relevant_idx + 1}:")
print(f"'{most_relevant_doc}'")
print(f"Relevance score: {similarities[most_relevant_idx]:.3f}\n")

# STEP 5: Generate answer using LLM + retrieved context
rag_prompt = f"""Based on the following context, answer the user's question.

Context: {most_relevant_doc}

Question: {user_question}

Answer:"""

print("STEP 5: LLM generates answer using retrieved context:")
print("\nPrompt sent to LLM:")
print(rag_prompt)
print("\nSimulated LLM Response:")
print("You have 30 days from the date of purchase to return items for a full refund. Please ensure items are in their original packaging and unused.")

### How RAG Works (Visual Summary)

```
Traditional LLM:
Question → LLM → Answer (may hallucinate if info not in training data)

RAG System:
Question → Vector Search → Retrieve Relevant Docs → LLM + Context → Accurate Answer
```

**Key Benefits:**
- ✅ Answers based on YOUR documents (not just training data)
- ✅ Reduces hallucinations (AI making up facts)
- ✅ Can cite sources ("According to document X...")
- ✅ Easily updateable (add new docs without retraining)

**Real-World Applications:**
- Customer support chatbots
- Legal document analysis
- Medical literature review
- Internal company knowledge bases

---
## Summary & Key Takeaways

Through these three examples, you've learned:

**1. Classical vs. Modern AI**
- Classical AI: Predicts categories from structured data (fast, cheap, narrow)
- Modern AI: Generates content, understands context, explains reasoning (flexible, powerful)
- Use case matters: Choose the right tool for the job

**2. Prompt Engineering**
- Quality of output depends on quality of prompts
- Key elements: Role, Context, Task, Constraints, Examples, Format
- Few-shot learning: Teach by example rather than explicit rules

**3. RAG (Retrieval-Augmented Generation)**
- Combines document search with LLM generation
- Grounds AI responses in your specific data
- Essential for enterprise AI applications

---

### Next Steps
- **Practice**: Try different prompting strategies with ChatGPT or Claude
- **Explore**: Build your own RAG system with LangChain or LlamaIndex
- **Experiment**: Compare costs and performance of different LLMs (GPT-4 vs GPT-3.5 vs Claude)

### Resources
- OpenAI Cookbook: https://cookbook.openai.com/
- LangChain Documentation: https://python.langchain.com/
- Prompt Engineering Guide: https://www.promptingguide.ai/